In [5]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed)
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(len(dataset))


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1


In [6]:
graph = dataset[0]
dataset[0].num_nodes
print (dataset.data)


HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [7]:

movies_features = graph["movie"].x
print (movies_features[:4])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [1]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 

In [ ]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
%run script/run.py -c config/transductive/inference.yaml --dataset WN18RR --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

In [2]:
%run script/run.py -c config/transductive/inference.yaml --dataset MovieLens100k --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
12:40:14   Random seed: 1024
12:40:14   Config file: config/transductive/inference.yaml
12:40:14   {'checkpoint': '/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth',
 'dataset': {'class': 'MovieLens100k', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func':

/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ultra/tasks.py:194: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  Ahh = torch.sparse.mm(EhT, Eh).coalesce()
12:40:19   MovieLens100k dataset
12:40:19   #train: 64000, #valid: 8000, #test: 8000


rel graph is built


12:40:19   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
12:40:19   Evaluate on valid


Load rspmm extension. This may take a while...


12:40:23   Note: NumExpr detected 28 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
12:40:23   NumExpr defaulting to 8 threads.
12:41:12   mr: 204.833
12:41:12   mrr: 0.0520594
12:41:12   hits@1: 0.0203125
12:41:12   hits@3: 0.0455
12:41:12   hits@10: 0.107125
12:41:12   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
12:41:12   Evaluate on test
12:42:05   mr: 201.8
12:42:05   mrr: 0.052182
12:42:05   hits@1: 0.0205625
12:42:05   hits@3: 0.04525
12:42:05   hits@10: 0.107


In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Now run your script
%run script/run.py -c config/transductive/inference.yaml --dataset MovieLens100k --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
12:28:41   Random seed: 1024
12:28:41   Config file: config/transductive/inference.yaml
12:28:41   {'checkpoint': '/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth',
 'dataset': {'class': 'MovieLens100k', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func':

/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ultra/tasks.py:194: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  Ahh = torch.sparse.mm(EhT, Eh).coalesce()
12:28:45   MovieLens100k dataset
12:28:45   #train: 64000, #valid: 8000, #test: 8000


rel graph is built


12:28:45   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
12:28:45   Evaluate on valid


Load rspmm extension. This may take a while...


/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [0,0,0], thread: [96,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [0,0,0], thread: [97,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [0,0,0], thread: [98,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [0,0,0], thread: [99,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: 

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


wandb: 🚀 View run run-MovieLens100k-2024-10-19_12-28 at: https://wandb.ai/pitri-eth-z-rich/tl4rec/runs/hxjrx3jm
wandb: Find logs at: wandb/run-20241019_122842-hxjrx3jm/logs


In [1]:
import numpy as np
import torch

print(np.__version__)  # Check NumPy version
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available


1.24.3
2.1.0
True


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [13]:
print(os.listdir())

['valid.txt', 'requirements.txt', 'MovieLense_load.ipynb', 'temp_pyg', 'config', '.git', '.gitignore', 'ultra', 'LICENSE', 'script', 'test.txt', 'train.txt', 'ckpts', 'asset', 'README.md', '.ipynb_checkpoints']


In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pitri (pitri-eth-z-rich). Use `wandb login --relogin` to force relogin


acc,▂▁▇▆▆▇█▇
loss,█▃▃▃▄▃▃▁
acc,0.83106
loss,0.0692
